# BO RDB v
* save, maxmemroy 반영 후
* throughtput 값 40000대부터 시작

* 모델 생성
* 목적함수 생성

In [2]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
import xgboost
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score
from lightgbm import LGBMRegressor, plot_importance
import lightgbm as lgb
from lightgbm import LGBMClassifier, LGBMRegressor

ModuleNotFoundError: No module named 'xgboost'

In [3]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


## Model 생성

### knobs 종류, 바운드 입력

In [ ]:
knobs_rdb = {
    
    "hash-max-ziplist-entries":     (256, 751), 
    "hash-max-ziplist-value": (16, 257),
#     "activerehashing": ('yes', 'no'),  ##yes
    "hz": (1, 41),
#     "dynamic-hz": ('yes', 'no'),

    "save0_0": (700,1100),
    "save0_1": (1,9),
    "save1_0": (100,500),
    "save1_1": (10,100),
    "save2_0": (30,90),
    "save2_1": (8000,12000),

#     "rdbcompression": ('yes', 'no'), ##yes
#     "rdbchecksum": ('yes', 'no'),    ##yes
#     "rdb-save-incremental-fsync": ('yes', 'no'),   ## yes

#     "activedefrag": ('yes','no'),  
    "active-defrag-threshold-lower": (1, 31),
    "active-defrag-threshold-upper": (70, 101),
    "active-defrag-cycle-min": (1, 31),
    "active-defrag-cycle-max": (70, 91),
    
    "maxmemory": (1000, 2900),   ##"volatile-lru"
#     "maxmemory-policy":     ("volatile-lru", "allkeys-lru", "volatile-lfu", "allkeys-lfu", 
#                              "volatile-random","allkeys-random", "volatile-ttl", "noeviction"),
    "maxmemory-samples": (3, 7),
    
#     "lazyfree-lazy-eviction": ('yes', 'no'),
#     "lazyfree-lazy-expire": ('yes', 'no'),
#     "lazyfree-lazy-server-del": ('yes', 'no')
}

knobs_list = list(knobs_rdb.keys())
knobs_bound = list(knobs_rdb.values())

In [ ]:
# 확인
print(knobs_list)
print(knobs_bound)

# 0. config data의 csv 파일 처리
csv 파일은 confile.conf 파일에서 선별한 knob 값들을 파싱하여 저장된 상태이다. 
* config_df 는 랜덤하게 값을 할당한 모든 변수 존재한다. 
* 부분적으로 knob 선별 작업 필요!

### csv 파일 가져오기

In [ ]:
# config data(.csv) -> DataFrame
config_df = pd.read_csv("./data/result_config0.csv", sep=',')

In [ ]:
# 확인
config_df.dtypes

### maxmemory 처리

In [ ]:
config_df['maxmemory']  = config_df['maxmemory'].str.strip('mb')

In [ ]:
config_df['maxmemory'] = pd.to_numeric(config_df['maxmemory'])

In [ ]:
# 확인
print(config_df['maxmemory'])
print(config_df.dtypes)

### null 값 처리

In [ ]:
# fillna(value = 0) 0 값으로 채워 넣기
config_df = config_df.fillna(value = 0)

In [ ]:
# 확인
## 확인 결과 모두 값이 0이면 제거하는 것이 좋다. !! 자동화 코드 만들 필요!
config_df.head()

### 카테고리 형 처리
* numeric
* category

애초에 위 두가지 형태 밖에 존재하지 않는다. numeric의 경우 모두 numeric으로 처리가 된 상태이며, category의 경우 object로 처리가 될것이다. 따라서 Object인 경우에만 category형태로 타입을 변경해준다.

In [ ]:
category_knob = []

for knob in config_df:
    if config_df[knob].dtype == 'O':  # Object인 경우 category 형으로 변경
        config_df[knob] = config_df[knob].astype('category')
        if knob in knobs_list:
            category_knob.append(knob)
#         category_knob.append(knob)

        
# print(category_knob)
# knob list에 없는 경우 제거
# for knob in category_knob:
#     print(knob)
    
#     if knob not in knobs_list:
#         print(knob)
#         category_knob.remove(knob)
        

In [ ]:
# 확인
category_knob

### knob 추출

In [ ]:
# knob 들만 추출
config_df_knob = config_df.loc[:,knobs_list]

In [ ]:
# 확인
config_df_knob.head()

### 모두 값이 0일 경우 제거

### result.csv 파일 가져오기

In [ ]:
# external data(.csv) -> DataFrame
external_df = pd.read_csv("./data/external_metrics0.csv", sep=',')
throughput_data = external_df['Gets_KB/sec']

In [ ]:
# throuput 값으로 예상 확인
external_df['Gets_KB/sec']

In [ ]:
print("max: {}".format(max(external_df['Gets_KB/sec'])))
print("min: {}".format(min(external_df['Gets_KB/sec'])))

In [ ]:
# 40000~ 50000대 확인
index1 = (external_df['Gets_KB/sec'] >= 47520.43)
index2 = (external_df['Gets_KB/sec'] <= 50000)

external_df[index2]

In [ ]:
# 60000대 확인
index3 = (external_df['Gets_KB/sec'] >= 65000)

external_df[index3]['Gets_KB/sec']

In [ ]:
import seaborn as sns
sns.boxplot(x = "Gets_KB/sec",data = external_df )
plt.show()

In [ ]:
# 히스토그램


### train, test 데이터 분리

In [ ]:
# train, test 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(config_df_knob, throughput_data ,test_size=0.2)

#lightGBM 파라미터 설정
fit_params={"early_stopping_rounds":10, 
            "eval_metric" : 'auc', 
            "eval_set" : [(X_test,y_test)],
            'eval_names': ['valid'],
            'verbose': 100,
            'feature_name': 'auto', # that's actually the default
            'categorical_feature': 'auto' # that's actually the default
           }

# 모델 생성
lgbr = lgb.LGBMRegressor(num_leaves= 15, max_depth=-1, 
                         random_state=314, 
                         silent=True, 
                         metric='None', 
                         n_jobs=4, 
                         n_estimators=1000,
                         colsample_bytree=0.9,
                         subsample=0.9,
                         learning_rate=0.1)

# 모델 fitting
lgbr.fit(X_train, y_train, **fit_params)

In [ ]:
# 확인
y_predict = lgbr.predict(X_test)

In [ ]:
print(lgbr)

---

# Bayeian Optimization 진행

### Objective Function 생성

In [ ]:
noise_level = 0.1

def objective_func(knob_value, noise_level=noise_level):
    knob_dict = {}

    for i, knob in enumerate(knobs_list): # 딕셔너리로 이동
        knob_dict[knob] = knob_value[i]
        
    if 'activedefrag' in knob_dict:
        if knob_dict['activedefrag'] == 'no':  # activedefrag no인 경우 나머지 관련 knob들에 0할당!
            activ_var = ['active-defrag-threshold-lower', 'active-defrag-threshold-upper', 'active-defrag-cycle-min', 'active-defrag-cycle-max']
            for activ_knob in activ_var:
                knob_dict[activ_knob] = 0     
    
    knob_df = pd.DataFrame(knob_dict, index=[0])  # 데이터 프레임으로 변환
    
    # 카테고리 변수 변경
    for knob in category_knob:
        knob_df[knob] = knob_df[knob].astype('category') 

    return -lgbr.predict(knob_df)[0]


#     change_numeric = ['active-defrag-threshold-lower', 'active-defrag-threshold-upper', 'active-defrag-cycle-min', 'active-defrag-cycle-max']


## gp_minimize 호출

In [ ]:
%matplotlib inline
print(__doc__)

import numpy as np
np.random.seed(237)
import matplotlib.pyplot as plt
from skopt.plots import plot_gaussian_process
from skopt import gp_minimize

In [ ]:
res = gp_minimize(objective_func,                  # the function to minimize
                  knobs_bound,      # the bounds on each dimension of x
                  acq_func="EI",      # the acquisition function
                  n_calls=50,         # the number of evaluations of f
                  n_random_starts=20,  # the number of random initialization points
                  noise=0.1**2,       # the noise level (optional)
                  random_state=1234)   # the random seed

In [ ]:
res.x

In [ ]:
res.fun # 63000

In [ ]:
knobs_rdb2 = {
    
    "hash-max-ziplist-entries":  445, 
    "hash-max-ziplist-value": 96,
    "activerehashing": ('yes',
    "hz": 11,
    "dynamic-hz": 'yes',

    "save0_0": 966,
    "save0_1": 2,
    "save1_0": 251,
    "save1_1": 11,
    "save2_0": 50,
    "save2_1": 9605,

    "rdbcompression": 'yes'
    "rdbchecksum": 'yes',
    "rdb-save-incremental-fsync": 'yes',

    "activedefrag": 'yes',
    "active-defrag-threshold-lower": 15,
    "active-defrag-threshold-upper": 90,
    "active-defrag-cycle-min": 31,
    "active-defrag-cycle-max": 81,
    
    "maxmemory": 1767,
    "maxmemory-policy":     ("volatile-lru", "allkeys-lru", "volatile-lfu", "allkeys-lfu", 
                             "volatile-random","allkeys-random", "volatile-ttl", "noeviction"),
    "maxmemory-samples": 3,
    
    "lazyfree-lazy-eviction": 'no',
    "lazyfree-lazy-expire": 'no',
    "lazyfree-lazy-server-del": 'no'
}

knobs_list = list(knobs_rdb.keys())
knobs_bound = list(knobs_rdb.values())